In [2]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.core import Workspace, Datastore, Environment, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.core import Pipeline, StepSequence
from azureml.widgets import RunDetails

ws = Workspace.from_config()
blob_store = Datastore(ws, "cat_test")    # workspaceblobstore is the default

# Create Experiment
experiment_name = 'infer_merge'
exp = Experiment(ws, experiment_name)

# Create Compute Target
# aml_compute_target = 'vdsm-cluster-d13'
aml_compute_target = 'vdsm-cluster-l80'
aml_compute = AmlCompute(ws, aml_compute_target)

env = Environment.get(workspace=ws,name="vdsm_all",version="1")

In [5]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.pipeline.steps import PythonScriptStep, ParallelRunStep, ParallelRunConfig
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.pipeline.core.graph import PipelineParameter

pipeline_run_config = ParallelRunConfig(environment=env,
                                        entry_script="infer_merge.py",
                                        error_threshold=-1,
                                        output_action="summary_only",
                                        compute_target=aml_compute,
                                        process_count_per_node=30,
                                        run_invocation_timeout=2400,
                                        node_count=3)

month_pipeline_param = PipelineParameter(name="month", default_value="2022-01")

ds_list = []
for i in range(1,249,2):
    ds_list.append(DatasetConsumptionConfig(name="row_{}_{}".format(i, i+1), dataset=ws.datasets.get("row_{}_{}".format(i, i+1))))

ds_list.append(DatasetConsumptionConfig(name="row_249", dataset=ws.datasets.get("row_249")))

merge_step = ParallelRunStep(
    name="merge_for_inference",
    inputs=ds_list,
    arguments=[
        "--month", month_pipeline_param
    ],
    parallel_run_config=pipeline_run_config)

trigger_step = PythonScriptStep(
    name="trigger_fusion_pipeline",
    script_name="trigger_fusion_pipeline.py",
    compute_target=aml_compute,
    source_directory="."
    )

step_sequence = StepSequence(steps=[merge_step, trigger_step])  

pipeline = Pipeline(workspace=ws, steps=step_sequence)
pipeline_run = exp.submit(pipeline)
print('VDSM training data fusion submitted for execution')
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion()


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/pipeline/core/_parallel_run_step_base.py:585: UserWarning: 
ParallelRunStep requires azureml-dataset-runtime[fuse,pandas] for tabular dataset.
Please add relevant package in CondaDependencies.
  UserWarning,
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


Created step merge_for_inference [1ab9d65e][b7f9ed19-9e81-41cc-9ea2-fbbc3494cc49], (This step will run and generate new outputs)
Created step trigger_fusion_pipeline [2b3f2428][0bea7444-8226-4ff3-89fa-bb3b8cd2e598], (This step will run and generate new outputs)
Submitted PipelineRun 36ee8e1d-544c-4539-ae71-73b72677a299
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/36ee8e1d-544c-4539-ae71-73b72677a299?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
VDSM training data fusion submitted for execution
PipelineRunId: 36ee8e1d-544c-4539-ae71-73b72677a299
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/36ee8e1d-544c-4539-ae71-73b72677a299?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
PipelineRun Status: NotStarted


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRun Status: Running


StepRunId: 6aee3e8e-01aa-48d2-8340-348360f3e9a9
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/6aee3e8e-01aa-48d2-8340-348360f3e9a9?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
StepRun( merge_for_inference ) Status: NotStarted
StepRun( merge_for_inference ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_c5de85262778c4dff5ed048dd2d1549f86f8991561f3a02b09c278a2ddae03b3_d.txt
2022-02-17T08:25:36Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/aml-ws-spacetech001/azureml/6aee3e8e-01aa-48d2-8340-348360f3e9a9/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/aml-ws-spacetech001/azureml/6aee3e8e-01aa-48d2-8340-348360f3e9a9/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=753832 -o nonempty -o allow_

'Finished'

In [4]:
# pipeline.publish_pipeline(name="infer_merge")
pipeline.publish(name="infer_merge", description="Merge S2 data for inference", version=8)

Created step merge_for_inference [a6d90ae3][47c22332-4a43-4490-82f1-321c55e4cd30], (This step will run and generate new outputs)Created step trigger_fusion_pipeline [6dd8d313][6e31d3b3-660e-442c-bc64-a1305e0e98a8], (This step will run and generate new outputs)



Pipeline(Name: infer_merge,
Id: 1543dd3b-e339-4ff8-96e4-e18aaf32fb57,
Status: Active,
Endpoint: https://australiaeast.api.azureml.ms/pipelines/v1.0/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourceGroups/arg-rt-pacedev-spacetech/providers/Microsoft.MachineLearningServices/workspaces/aml-ws-spacetech001/PipelineRuns/PipelineSubmit/1543dd3b-e339-4ff8-96e4-e18aaf32fb57)